In [23]:
import f3dasm
import numpy as np
from dataclasses import dataclass
from typing import List, Any
from hydra.core.config_store import ConfigStore
import hydra

In [24]:
from f3dasm.run_optimization import execute
execute()

ImportError: cannot import name 'execute' from 'f3dasm.run_optimization' (/home/martin/Documents/GitHub/F3DASM/f3dasm/run_optimization.py)

In [9]:
def find_class(module, query: str):
    return getattr(module, query)


In [10]:
a = getattr(f3dasm,'sampling')

getattr(a,'RandomUniformSampling')

f3dasm.sampling.samplers.RandomUniformSampling

In [11]:
@dataclass
class Block:
    type: str
    implementation: str
    arguments: dict

    def create(self):
        # Select correct block-type
        block_type = find_class(f3dasm, self.type)

        # Select correct implementation
        implementation_type = find_class(block_type, self.implementation)
        
        # Create implementation with arguments
        return implementation_type(**self.arguments)

    def __call__(self, data: f3dasm.Data):
        # Create block
        mod = self.create()

        # Execute the block
        results = mod(data)

        # Return results
        return results

In [12]:
@dataclass
class DesignConfig:
    lower_bound: float
    upper_bound: float
    dimensionality: int

@dataclass
class FunctionConfig:
    function_name: str
    noise: float or Any

@dataclass
class OptimizerConfig:
    optimizer_name: str


@dataclass
class SamplerConfig:
    sampler_name: str

@dataclass
class ExecutionConfig:
    iterations: int
    realizations: int

@dataclass
class Config:
    design: DesignConfig
    function: FunctionConfig
    sampler: SamplerConfig
    execution: ExecutionConfig

In [13]:
cs = ConfigStore.instance()
cs.store(name="f3dasm_config", node=Config)

@hydra.main(config_path=".", config_name="config")
def main(cfg: Config):
    print(cfg)

cs

In [15]:
main()

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: argument --shell-completion/-sc: ignored explicit argument '9002'


AssertionError: 

In [ ]:
function_name = 'Ackley'
optimizer_name = 'CMAES'
sampler_name = 'RandomUniformSampling'
dimensionality = 2
iterations = 10
realizations = 4
bounds = np.tile([-1.0, 1.0], (dimensionality, 1))
noise = False
hyperparameters = {}

In [ ]:
function_class: f3dasm.Function = find_class(function_name, f3dasm.functions)
optimizer_class: f3dasm.Optimizer = find_class(optimizer_name, f3dasm.optimization)
sampler_class: f3dasm.SamplingInterface = find_class(sampler_name, f3dasm.sampling)

In [ ]:
design = f3dasm.make_nd_continuous_design(bounds=bounds, dimensionality=dimensionality)
data = f3dasm.Data(design=design)

In [ ]:
function = function_class(dimensionality=dimensionality, noise=noise)
optimizer = optimizer_class(data=data, hyperparameters=hyperparameters)
sampler = sampler_class(design=data.design)

In [ ]:
args = {'optimizer': optimizer,
'function': function,
'sampler': sampler,
'iterations': iterations,
'realizations': realizations}

In [ ]:
result = f3dasm.run_multiple_realizations(**args)
result[0].data